<a href="https://colab.research.google.com/github/rakesh19889/Ai-task/blob/main/SecqurAIse_Technologies_Private_Limited.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install opencv-python numpy



In [5]:
import cv2
import numpy as np
import time
import csv

# Function to get the current quadrant of the ball
def get_quadrant(x, y, width, height):
    if x < width // 2:
        if y < height // 2:
            return 1
        else:
            return 3
    else:
        if y < height // 2:
            return 2
        else:
            return 4

# Function to detect and track colored balls
def track_balls(video_path, output_csv):
    # Open video file
    cap = cv2.VideoCapture(video_path)

    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Define the color ranges for detection
    color_ranges = {
        'red': ((0, 50, 50), (10, 255, 255)),
        'green': ((50, 50, 50), (70, 255, 255)),
        'blue': ((110, 50, 50), (130, 255, 255))
    }

    # To keep track of the last known quadrant of each ball
    last_quadrant = {}

    # Open a CSV file to write the events
    with open(output_csv, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['Time', 'Quadrant Number', 'Ball Colour', 'Type'])

        frame_count = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Convert frame to HSV color space
            hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

            # Iterate over each color to detect balls
            for color, (lower, upper) in color_ranges.items():
                mask = cv2.inRange(hsv, lower, upper)
                mask = cv2.erode(mask, None, iterations=2)
                mask = cv2.dilate(mask, None, iterations=2)

                # Find contours of the masked regions
                contours, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                for contour in contours:
                    if cv2.contourArea(contour) > 100:  # Minimum area to filter noise
                        ((x, y), radius) = cv2.minEnclosingCircle(contour)
                        quadrant = get_quadrant(x, y, width, height)
                        ball_id = f"{color}_ball"

                        # Check if the ball has changed quadrant
                        if ball_id not in last_quadrant:
                            last_quadrant[ball_id] = quadrant
                            event_type = 'Entry'
                        elif last_quadrant[ball_id] != quadrant:
                            event_type = 'Exit'
                            last_quadrant[ball_id] = quadrant

                        # Record the event
                        timestamp = frame_count / fps
                        csvwriter.writerow([timestamp, quadrant, color, event_type])

            frame_count += 1

    cap.release()

# Example usage
track_balls('/content/drive/MyDrive/Untitled folder/AI Assignment video.mp4', 'output_events.csv')
